In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('train.csv')

In [4]:
train.sample(5)

,id,keyword,location,text,target
6632,9499,terrorist,NaN,HereÛªs how media in Pakistan covered the cap...,1
1335,1929,burning%20buildings,NaN,@fewmoretweets all lives matter. Just not a fa...,1
3181,4567,emergency%20plan,Surrey & Manchester,NHS England announces new plan to meet emergen...,0
15,23,NaN,NaN,What's up man?,0
5225,7467,obliteration,828/704(Soufside)/while looking goofy in NJ,Back in 02 to 03 would never said that 50 woul...,0


In [10]:
train['keyword'].fillna('', inplace=True)

In [14]:
X, y = train.drop('target', axis=1), train['target']

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_text = CountVectorizer()
vectorizer_keyword = CountVectorizer()

X_train_text = vectorizer_text.fit_transform(X_train['text'])
X_train_keyword = vectorizer_keyword.fit_transform(X_train['keyword'])
X_test_text = vectorizer_text.transform(X_test['text'])
X_test_keyword = vectorizer_keyword.transform(X_test['keyword'])

In [22]:
from scipy import sparse

X_train = sparse.hstack([X_train_keyword, X_train_text])
X_test = sparse.hstack([X_test_keyword, X_test_text])

In [30]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train, y_train)

MultinomialNB()

In [31]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       869
           1       0.78      0.70      0.74       654

    accuracy                           0.79      1523
   macro avg       0.78      0.78      0.78      1523
weighted avg       0.79      0.79      0.78      1523



In [32]:
model = MultinomialNB()

X = sparse.vstack([X_train, X_test])
y = pd.concat([y_train, y_test], axis=0)
model.fit(X, y)

MultinomialNB()

In [34]:
test = pd.read_csv('test.csv')
test['keyword'].fillna('', inplace=True)

X_test = sparse.hstack([
    vectorizer_keyword.transform(test['keyword']),
    vectorizer_text.transform(test['text'])
])

/var/folders/v5/gy4nc82s2w94_ymtt7vqyxdc0000gn/T/ipykernel_8202/1590111771.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['keyword'].fillna('', inplace=True)


In [35]:
y_test = model.predict(X_test)

In [39]:
test

,id,keyword,location,text
0,0,,NaN,Just happened a terrible car crash
1,2,,NaN,"Heard about #earthquake is different cities, s..."
2,3,,NaN,"there is a forest fire at spot pond, geese are..."
3,9,,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [43]:
result = pd.concat([test['id'], pd.DataFrame(y_test, columns=['target'])], axis=1).set_index('id')
result.to_csv('submission.csv')